In [9]:
# import
import numpy as np
import tensor.tensor_product_wrapper as tp
from utils.plotting_utils import montage_array, slice_subplots, classification_plots
import matplotlib.pyplot as plt
import similarity_metrics as sm
import scipy.io
import utils.starplus_utils as starp
from numpy.linalg import norm
from sklearn.model_selection import train_test_split
import transform_matrices as tm
import pandas as pd

import os
import pickle

In [10]:
# saving options
save = False
filename = 'local_tsvdm'

# plotting options
plot = False

In [11]:
# choose product type {'m'}
prod_type = 'm'
ortho = True

In [12]:
def projection(A, U, prod_type='m', M=None, dim_list=(), ortho=True):
    training_coeff = tp.ten_prod(tp.ten_tran(U, prod_type=prod_type), A, prod_type=prod_type, M=M, dim_list=dim_list,
                                 ortho=ortho)
    return tp.ten_prod(U, training_coeff, prod_type=prod_type, M=M, dim_list=dim_list, ortho=ortho)


In [13]:
# for reproducibility
np.random.seed(20)

column_names = ['ROI #','ROI Name','k','accuracy']
accuracy_df = pd.DataFrame(columns=column_names)

In [14]:
# load data
# subject_ids = ['04847', '04799', '04820', '05675', '05710']
subject_ids = ['04847']
tensor_PS = np.empty([64, 64, 8, 16, 80])
labels = np.empty(0)
for id in subject_ids:
    star_plus_data = scipy.io.loadmat('data/data-starplus-' + id + '-v7.mat')
    tensor_PS_id, labels_id = starp.get_labels(star_plus_data)
    tensor_PS = np.concatenate((tensor_PS, tensor_PS_id), axis=-1)
    labels = np.concatenate((labels, labels_id.reshape(-1)))

    star_plus_data = scipy.io.loadmat('data/data-starplus-' + id + '-v7.mat')
    roi_tensor, my_color_map, names = starp.visualize_roi(star_plus_data)

tensor_PS = tensor_PS[:, :, :, :, 80:] / norm(tensor_PS)



In [15]:
# split
training_data, test_data, training_labels, test_labels = train_test_split(np.moveaxis(tensor_PS, -1, 0), labels,
                                                                          test_size=0.33, random_state=42)
training_data = np.moveaxis(training_data, 0, 1)
test_data = np.moveaxis(test_data, 0, 1)

roi_tensor = np.repeat(np.repeat(roi_tensor.reshape(64, 64, 8, 1, 1), 16, axis=3), training_data.shape[1], axis=4)
roi_tensor = np.moveaxis(np.moveaxis(roi_tensor, -1, 0), 0, 1)

print('Loaded data...')

Loaded data...


In [16]:
# choose transformations for M
for roi in range(0,len(names)):
    M = []
    for i in range(2, training_data.ndim):
        if i == 2:
            M.append(tm.roi_left_singular_matrix(training_data, i, roi_tensor, roi))
        else:
            M.append(tm.haar_matrix(training_data.shape[i], normalized=True))
    
    print('Formed M...')
    # ==================================================================================================================== #
    #%%
    # form local t-svd
    num_classes = len(np.unique(training_labels))
    range_k = np.arange(4,5,1)
    
    U = []
    for i in range(num_classes):
        u, _, _, _ = tp.ten_svd(training_data[:, training_labels == i], k=max(range_k), prod_type='m', M=M, ortho=ortho)
        U.append(u)
    
    print('Formed local t-SVDM...')
    # ==================================================================================================================== #
    # compute results on training and test data
    training_error = np.zeros([num_classes, training_data.shape[1], len(range_k)])
    test_error = np.zeros([num_classes, test_data.shape[1], len(range_k)])
    for j, k in enumerate(range_k):
        for i in range(num_classes):
            training_projection = projection(training_data, U[i][:, :k], prod_type='m', M=M)
            training_error[i, :, j] = sm.frobenius_metric(training_data, training_projection, axis=1)
    
            test_projection = projection(test_data, U[i][:, :k], prod_type='m', M=M)
            test_error[i, :, j] = sm.frobenius_metric(test_data, test_projection, axis=1)
    
    
    # classification
    training_predicted_classes = np.argmin(training_error, axis=0).reshape(-1, len(range_k))
    test_predicted_classes = np.argmin(test_error, axis=0).reshape(-1, len(range_k))
    
    # results
    training_num_correct = np.sum(training_predicted_classes == training_labels.reshape(-1, 1), axis=0)
    training_accuracy = training_num_correct / training_data.shape[1]
    
    test_num_correct = np.sum(test_predicted_classes == test_labels.reshape(-1, 1), axis=0)
    test_accuracy = test_num_correct / test_data.shape[1]
    
    for j, k in enumerate(range_k):
        print('k = %d: train accuracy = %0.2f\ttest accuracy = %0.2f' %
              (k, 100 * training_accuracy[j], 100 * test_accuracy[j]))
    print(roi)
    roi_num_list = [roi] * len(range_k)
    roi_name_list = [names[roi]] * len(range_k)
    accuracy = pd.DataFrame(
    {'ROI #' : roi_num_list,
    'ROI Name' : roi_name_list,
    'k': range_k,
    'accuracy': test_accuracy}
    )
    accuracy_df = accuracy_df.append(accuracy)

Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 85.19
0
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 85.19
1
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 74.07
2
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 77.78
3
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 81.48
4
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 85.19
5
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 85.19
6
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 77.78
7
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 77.78
8
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 81.48
9
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 81.48
10
Formed M.

In [17]:
print(accuracy_df)
accuracy_df_ordered = accuracy_df.sort_values(by='accuracy')
maximums = accuracy_df_ordered.tail(13)
maximums_list = maximums['ROI #'].tolist()
print(maximums_list)

  ROI # ROI Name  k  accuracy
0     0     CALC  4  0.851852
0     1   LDLPFC  4  0.851852
0     2     LFEF  4  0.740741
0     3     LIFG  4  0.777778
0     4     LIPL  4  0.814815
0     5     LIPS  4  0.851852
0     6      LIT  4  0.851852
0     7    LOPER  4  0.777778
0     8   LPPREC  4  0.777778
0     9     LSGA  4  0.814815
0    10     LSPL  4  0.814815
0    11       LT  4  0.740741
0    12    LTRIA  4  0.851852
0    13   RDLPFC  4  0.851852
0    14     RFEF  4  0.814815
0    15     RIPL  4  0.814815
0    16     RIPS  4  0.777778
0    17      RIT  4  0.851852
0    18    ROPER  4  0.777778
0    19   RPPREC  4  0.777778
0    20     RSGA  4  0.777778
0    21     RSPL  4  0.777778
0    22       RT  4  0.814815
0    23    RTRIA  4  0.777778
0    24      SMA  4  0.777778
[9, 10, 4, 22, 14, 15, 0, 13, 6, 5, 1, 17, 12]


In [18]:
print('Here are the ROIs that achieved maximum classification accuracy when applied to i==2 dimension (Haar matrix for all other dimensions):')
print(maximums)

Here are the ROIs that achieved maximum classification accuracy when applied to i==2 dimension (Haar matrix for all other dimensions):
  ROI # ROI Name  k  accuracy
0     9     LSGA  4  0.814815
0    10     LSPL  4  0.814815
0     4     LIPL  4  0.814815
0    22       RT  4  0.814815
0    14     RFEF  4  0.814815
0    15     RIPL  4  0.814815
0     0     CALC  4  0.851852
0    13   RDLPFC  4  0.851852
0     6      LIT  4  0.851852
0     5     LIPS  4  0.851852
0     1   LDLPFC  4  0.851852
0    17      RIT  4  0.851852
0    12    LTRIA  4  0.851852


In [19]:
accuracy_df_2 = pd.DataFrame(columns=column_names)
# choose transformations for M (using roi matrix on all dimensions)
for roi in maximums_list:
    M = []
    for i in range(2, training_data.ndim):
        M.append(tm.roi_left_singular_matrix(training_data, i, roi_tensor, roi))
    
    print('Formed M...')
    # ==================================================================================================================== #
    #%%
    # form local t-svd
    num_classes = len(np.unique(training_labels))
    range_k = np.arange(4,5,1)
    
    U = []
    for i in range(num_classes):
        u, _, _, _ = tp.ten_svd(training_data[:, training_labels == i], k=max(range_k), prod_type='m', M=M, ortho=ortho)
        U.append(u)
    
    print('Formed local t-SVDM...')
    # ==================================================================================================================== #
    # compute results on training and test data
    training_error = np.zeros([num_classes, training_data.shape[1], len(range_k)])
    test_error = np.zeros([num_classes, test_data.shape[1], len(range_k)])
    for j, k in enumerate(range_k):
        for i in range(num_classes):
            training_projection = projection(training_data, U[i][:, :k], prod_type='m', M=M)
            training_error[i, :, j] = sm.frobenius_metric(training_data, training_projection, axis=1)
    
            test_projection = projection(test_data, U[i][:, :k], prod_type='m', M=M)
            test_error[i, :, j] = sm.frobenius_metric(test_data, test_projection, axis=1)
    
    
    # classification
    training_predicted_classes = np.argmin(training_error, axis=0).reshape(-1, len(range_k))
    test_predicted_classes = np.argmin(test_error, axis=0).reshape(-1, len(range_k))
    
    # results
    training_num_correct = np.sum(training_predicted_classes == training_labels.reshape(-1, 1), axis=0)
    training_accuracy = training_num_correct / training_data.shape[1]
    
    test_num_correct = np.sum(test_predicted_classes == test_labels.reshape(-1, 1), axis=0)
    test_accuracy = test_num_correct / test_data.shape[1]
    
    for j, k in enumerate(range_k):
        print('k = %d: train accuracy = %0.2f\ttest accuracy = %0.2f' %
              (k, 100 * training_accuracy[j], 100 * test_accuracy[j]))
    print(roi)
    roi_num_list = [roi] * len(range_k)
    roi_name_list = [names[roi]] * len(range_k)
    accuracy = pd.DataFrame(
    {'ROI #' : roi_num_list,
    'ROI Name' : roi_name_list,
    'k': range_k,
    'accuracy': test_accuracy}
    )
    accuracy_df_2 = accuracy_df_2.append(accuracy)

Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 77.78
9
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 74.07
10
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 70.37
4
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 77.78
22
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 70.37
14
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 70.37
15
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 77.78
0
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 77.78
13
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 77.78
6
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 70.37
5
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 100.00	test accuracy = 70.37
1
Forme

In [21]:
print(accuracy_df_2)
accuracy_df_2_ordered = accuracy_df_2.sort_values(by='accuracy')
print('Here are the ROIs that achieved maximum classification accuracy when applied to all dimensions:')
print(accuracy_df_2_ordered)

  ROI # ROI Name  k  accuracy
0     9     LSGA  4  0.777778
0    10     LSPL  4  0.740741
0     4     LIPL  4  0.703704
0    22       RT  4  0.777778
0    14     RFEF  4  0.703704
0    15     RIPL  4  0.703704
0     0     CALC  4  0.777778
0    13   RDLPFC  4  0.777778
0     6      LIT  4  0.777778
0     5     LIPS  4  0.703704
0     1   LDLPFC  4  0.703704
0    17      RIT  4  0.703704
0    12    LTRIA  4  0.814815
Here are the ROIs that achieved maximum classification accuracy when applied to all dimensions:
  ROI # ROI Name  k  accuracy
0     4     LIPL  4  0.703704
0    14     RFEF  4  0.703704
0    15     RIPL  4  0.703704
0     5     LIPS  4  0.703704
0     1   LDLPFC  4  0.703704
0    17      RIT  4  0.703704
0    10     LSPL  4  0.740741
0     9     LSGA  4  0.777778
0    22       RT  4  0.777778
0     0     CALC  4  0.777778
0    13   RDLPFC  4  0.777778
0     6      LIT  4  0.777778
0    12    LTRIA  4  0.814815


In [22]:
accuracy_df_3 = pd.DataFrame(columns=column_names)
# choose transformations for M
for roi in range(0,len(names)):
    M = []
    for i in range(2, training_data.ndim):
        if i == 2:
            M.append(tm.roi_left_singular_matrix(training_data, i, roi_tensor, roi))
        else:
            M.append(tm.banded_matrix(training_data.shape[i], 2)) #is this the right banded matrix?
    
    print('Formed M...')
    # ==================================================================================================================== #
    #%%
    # form local t-svd
    num_classes = len(np.unique(training_labels))
    range_k = np.arange(4,5,1)
    
    U = []
    for i in range(num_classes):
        u, _, _, _ = tp.ten_svd(training_data[:, training_labels == i], k=max(range_k), prod_type='m', M=M, ortho=ortho)
        U.append(u)
    
    print('Formed local t-SVDM...')
    # ==================================================================================================================== #
    # compute results on training and test data
    training_error = np.zeros([num_classes, training_data.shape[1], len(range_k)])
    test_error = np.zeros([num_classes, test_data.shape[1], len(range_k)])
    for j, k in enumerate(range_k):
        for i in range(num_classes):
            training_projection = projection(training_data, U[i][:, :k], prod_type='m', M=M)
            training_error[i, :, j] = sm.frobenius_metric(training_data, training_projection, axis=1)
    
            test_projection = projection(test_data, U[i][:, :k], prod_type='m', M=M)
            test_error[i, :, j] = sm.frobenius_metric(test_data, test_projection, axis=1)
    
    
    # classification
    training_predicted_classes = np.argmin(training_error, axis=0).reshape(-1, len(range_k))
    test_predicted_classes = np.argmin(test_error, axis=0).reshape(-1, len(range_k))
    
    # results
    training_num_correct = np.sum(training_predicted_classes == training_labels.reshape(-1, 1), axis=0)
    training_accuracy = training_num_correct / training_data.shape[1]
    
    test_num_correct = np.sum(test_predicted_classes == test_labels.reshape(-1, 1), axis=0)
    test_accuracy = test_num_correct / test_data.shape[1]
    
    for j, k in enumerate(range_k):
        print('k = %d: train accuracy = %0.2f\ttest accuracy = %0.2f' %
              (k, 100 * training_accuracy[j], 100 * test_accuracy[j]))
    print(roi)
    roi_num_list = [roi] * len(range_k)
    roi_name_list = [names[roi]] * len(range_k)
    accuracy = pd.DataFrame(
    {'ROI #' : roi_num_list,
    'ROI Name' : roi_name_list,
    'k': range_k,
    'accuracy': test_accuracy}
    )
    accuracy_df_3 = accuracy_df_3.append(accuracy)

Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 45.28	test accuracy = 59.26
0
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 45.28	test accuracy = 55.56
1
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 37.74	test accuracy = 55.56
2
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 41.51	test accuracy = 51.85
3
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 43.40	test accuracy = 51.85
4
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 47.17	test accuracy = 51.85
5
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 47.17	test accuracy = 48.15
6
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 47.17	test accuracy = 51.85
7
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 43.40	test accuracy = 48.15
8
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 41.51	test accuracy = 51.85
9
Formed M...
Formed local t-SVDM...
k = 4: train accuracy = 47.17	test accuracy = 51.85
10
Formed M...
Formed l

In [23]:
print(accuracy_df_3)
accuracy_df_3_ordered = accuracy_df_3.sort_values(by='accuracy')
print('Here are the ROIs that achieved maximum classification accuracy when applied to i==2 dimension (banded matrix for other dimensions):')
print(accuracy_df_3_ordered)

  ROI # ROI Name  k  accuracy
0     0     CALC  4  0.592593
0     1   LDLPFC  4  0.555556
0     2     LFEF  4  0.555556
0     3     LIFG  4  0.518519
0     4     LIPL  4  0.518519
0     5     LIPS  4  0.518519
0     6      LIT  4  0.481481
0     7    LOPER  4  0.518519
0     8   LPPREC  4  0.481481
0     9     LSGA  4  0.518519
0    10     LSPL  4  0.518519
0    11       LT  4  0.481481
0    12    LTRIA  4  0.481481
0    13   RDLPFC  4  0.407407
0    14     RFEF  4  0.592593
0    15     RIPL  4  0.518519
0    16     RIPS  4  0.518519
0    17      RIT  4  0.481481
0    18    ROPER  4  0.518519
0    19   RPPREC  4  0.518519
0    20     RSGA  4  0.518519
0    21     RSPL  4  0.518519
0    22       RT  4  0.518519
0    23    RTRIA  4  0.555556
0    24      SMA  4  0.518519
Here are the ROIs that achieved maximum classification accuracy when applied to i==2 dimension (banded matrix for other dimensions):
  ROI # ROI Name  k  accuracy
0    13   RDLPFC  4  0.407407
0    12    LTRIA  4  0.4814